In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
import warnings
warnings.filterwarnings('ignore')
import math
import random

In [2]:
pitcher = pd.read_csv('private_pitcher.csv',encoding='CP949')
batter = pd.read_csv('private_batter.csv', encoding='CP949')
batter['GDAY_DS'] = pd.to_datetime(batter['GDAY_DS'],format='%Y-%m-%d')
pitcher['GDAY_DS'] = pd.to_datetime(pitcher['GDAY_DS'],format='%Y-%m-%d')

# 1. 모델 적용(지수평활법)

- 각 선수의 경기기록이 10 미만인 경우 모델을 사용하지 않고 평균값을 사용

In [3]:
# row가 10이상인 case만 추출
temp1 = pitcher.head(1)

for i in pitcher.P_ID.unique():
    df = pitcher[pitcher['P_ID']==i]
    num = int(df.count()[1])
    if num>=10: # 10 이상인 경우만 추출
         temp1 = temp1.append(pitcher[pitcher['P_ID']==i])

In [4]:
temp2 = batter.head(1)

for i in batter.P_ID.unique():
    df = batter[batter['P_ID']==i]
    num = int(df.count()[1])
    if num>=10:
         temp2 = temp2.append(batter[batter['P_ID']==i])

In [5]:
pitcher_pred = pd.DataFrame(columns=temp1.drop(['GDAY_DS','ER','ERA','BF','AB','HIT','KK','SLG'],axis=1).columns)
batter_pred = pd.DataFrame(columns=temp2.drop(['GDAY_DS','HIT','AVG'],axis=1).columns)

In [6]:
pitcher_pred['P_ID'] = temp1.P_ID.unique()
batter_pred['P_ID'] = temp2.P_ID.unique()

In [7]:
pitcher_pred.head()

,T_ID,P_ID,TB_SC,INN2,PA-AB,H1,H2,H3,HR,SB_SR,WP,BABIP,KK9,BB9
0,NaN,60146,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,61891,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,62146,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,62698,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,65764,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
batter_pred.head()

,T_ID,P_ID,TB_SC,PA-AB,AB,RUN,RBI,SH+SF,KK,SB_trial,BABIP
0,NaN,60404,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,61102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,61186,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,62164,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,62700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- 이제 x값을 채워주자

In [9]:
# x변수 이름
col1 = np.array(pitcher.drop(['GDAY_DS','T_ID','P_ID','TB_SC','ER','ERA','BF','AB','HIT','KK','SLG'],axis=1).columns) #우선 명목형 변수 제거함..
col2 = np.array(batter.drop(['GDAY_DS','T_ID','P_ID','TB_SC','HIT','AVG'],axis=1).columns)

In [10]:
for i in col1:
    value = []
    for j in pitcher_pred.P_ID:
        df = temp1[temp1['P_ID']==j][['GDAY_DS',i]]
        fit = SimpleExpSmoothing(np.array(df[i])).fit(smoothing_level=0.1) # 모델 적용
        pred = float(fit.forecast(1)) 
        value.append(round(pred,2)) # 예측값을 반올림해줌
    pitcher_pred[i] = value

In [11]:
pitcher_pred.head()

,T_ID,P_ID,TB_SC,INN2,PA-AB,H1,H2,H3,HR,SB_SR,WP,BABIP,KK9,BB9
0,NaN,60146,NaN,2.47,0.60,0.60,0.14,0.00,0.18,0.10,0.01,0.29,8.40,6.90
1,NaN,61891,NaN,2.58,0.56,0.56,0.11,0.05,0.03,0.00,0.01,0.36,10.77,7.05
2,NaN,62146,NaN,2.22,0.31,0.26,0.11,0.06,0.05,0.00,0.02,0.29,10.72,2.87
3,NaN,62698,NaN,17.73,1.74,4.04,0.67,0.01,1.02,0.49,0.19,0.29,8.56,2.19
4,NaN,65764,NaN,14.12,2.70,3.72,0.63,0.01,0.62,0.54,0.20,0.31,7.92,5.09


In [12]:
for i in col2:
    value = []
    for j in batter_pred.P_ID:
        df = temp2[temp2['P_ID']==j][['GDAY_DS',i]]
        fit = SimpleExpSmoothing(np.array(df[i])).fit(smoothing_level=0.1) # 모델 적용
        pred = float(fit.forecast(1)) 
        value.append(round(pred,2)) # 예측값을 반올림해줌
    batter_pred[i] = value

In [13]:
batter_pred.head()

,T_ID,P_ID,TB_SC,PA-AB,AB,RUN,RBI,SH+SF,KK,SB_trial,BABIP
0,NaN,60404,NaN,0.10,1.19,0.08,0.01,0.03,0.35,0.08,0.06
1,NaN,61102,NaN,0.12,3.73,0.21,0.47,0.01,0.98,0.00,0.27
2,NaN,61186,NaN,0.83,3.62,0.73,0.52,0.11,0.47,0.11,0.33
3,NaN,62164,NaN,0.26,1.82,0.17,0.16,0.03,0.75,0.01,0.20
4,NaN,62700,NaN,0.15,3.52,0.50,0.21,0.01,0.94,0.09,0.38


# 2. sample이 10 미만인 경우 평균 값으로 채워주자

In [14]:
# row가 10미만인 case 추출
temp3 = pd.DataFrame()
for i in pitcher.P_ID.unique():
    df = pitcher[pitcher['P_ID']==i]
    num = int(df.count()[1])
    if num<10: # 10 이상인 경우만 추출
         temp3 = temp3.append(pitcher[pitcher['P_ID']==i])

In [15]:
temp4 = pd.DataFrame()
for i in batter.P_ID.unique():
    df = batter[batter['P_ID']==i]
    num = int(df.count()[1])
    if num<10:
         temp4 = temp4.append(batter[batter['P_ID']==i])

In [16]:
pitcher_pred2 = pd.DataFrame(columns=temp3.drop(['GDAY_DS','ER','ERA','BF','AB','HIT','KK','SLG'],axis=1).columns)
batter_pred2 = pd.DataFrame(columns=temp4.drop(['GDAY_DS','HIT','AVG'],axis=1).columns)

In [17]:
pitcher_pred2['P_ID'] = temp3.P_ID.unique()
batter_pred2['P_ID'] = temp4.P_ID.unique()

In [18]:
for i in col1:
    value = []
    for j in pitcher_pred2.P_ID:
        df = temp3[temp3['P_ID']==j][['GDAY_DS',i]]
        pred = df[i].mean()
        value.append(round(pred,2)) # 예측값을 반올림해줌
    pitcher_pred2[i] = value

In [19]:
pitcher_pred2.head()

,T_ID,P_ID,TB_SC,INN2,PA-AB,H1,H2,H3,HR,SB_SR,WP,BABIP,KK9,BB9
0,NaN,66440,NaN,12.33,5.00,5.00,1.33,0.00,0.33,0.17,0.67,0.40,3.57,10.56
1,NaN,66748,NaN,9.56,4.44,3.33,0.11,0.22,0.44,0.61,0.67,0.42,5.50,19.67
2,NaN,71851,NaN,9.67,1.33,1.67,1.00,0.00,0.33,0.00,0.00,0.25,5.77,3.45
3,NaN,75268,NaN,5.33,1.33,2.00,1.33,0.00,0.67,0.00,0.00,0.50,6.43,5.14
4,NaN,61569,NaN,6.00,0.00,2.00,0.00,0.00,1.00,0.00,0.00,0.29,4.50,0.00


In [20]:
for i in col2:
    value = []
    for j in batter_pred2.P_ID:
        df = temp4[temp4['P_ID']==j][['GDAY_DS',i]]
        pred = df[i].mean()
        value.append(round(pred,2)) # 예측값을 반올림해줌
    batter_pred2[i] = value

In [21]:
batter_pred2.head()

,T_ID,P_ID,TB_SC,PA-AB,AB,RUN,RBI,SH+SF,KK,SB_trial,BABIP
0,NaN,62056,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,NaN,61643,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,NaN,60757,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,NaN,61891,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,NaN,65546,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 3. 1 2번에서 구한 데이터 프레임 concat

In [22]:
batter_pred = batter_pred.append(batter_pred2)
pitcher_pred = pitcher_pred.append(pitcher_pred2)

### 1)  2020년 경기기록이 없는 경우 제외

In [23]:
total = pitcher['P_ID'].unique()
recent = pitcher[pitcher['GDAY_DS'].dt.year==2020]['P_ID'].unique()
a = np.setdiff1d(total,recent)

In [24]:
for i in a:
    ind = pitcher_pred[pitcher_pred['P_ID']==i].index
    pitcher_pred = pitcher_pred.drop(ind)

In [25]:
total = batter['P_ID'].unique()
recent = batter[batter['GDAY_DS'].dt.year==2020]['P_ID'].unique()
a = np.setdiff1d(total,recent)

In [26]:
for i in a:
    ind = batter_pred[batter_pred['P_ID']==i].index
    batter_pred = batter_pred.drop(ind)

### 2) 2020년 소속된 팀 값 채워주기

- 2020년에 이직한 선수가 있음을 발견.. 가장 최근에 소속된 팀으로 값 지정

In [27]:
a = pitcher[pitcher['GDAY_DS'].dt.year==2020][['T_ID','P_ID']]
who = []
for i in a.P_ID:
    num = a[a['P_ID']==i]['T_ID'].nunique()
    if num>1 : who.append(i)

In [28]:
set(who)

{60768, 61643}

In [29]:
case1 = pitcher_pred[(pitcher_pred['P_ID']==60768)]
case2 = pitcher_pred[(pitcher_pred['P_ID']==61643)]
case1.T_ID = 'SK'
case2.T_ID = 'OB'
pitcher_pred = pitcher_pred.drop(pitcher_pred[(pitcher_pred['P_ID']==60768)].index)
pitcher_pred = pitcher_pred.drop(pitcher_pred[(pitcher_pred['P_ID']==61643)].index)
# 60768 :Sk 61643 OB

In [30]:
team = []
for i in pitcher_pred.P_ID:
    t = pitcher[(pitcher['P_ID']==i)&(pitcher['GDAY_DS'].dt.year==2020)]['T_ID'].unique().item()
    team.append(t)
pitcher_pred['T_ID'] = team

In [31]:
pitcher_pred = pitcher_pred.append(case1)
pitcher_pred = pitcher_pred.append(case2)

In [32]:
pitcher_pred.head()

,T_ID,P_ID,TB_SC,INN2,PA-AB,H1,H2,H3,HR,SB_SR,WP,BABIP,KK9,BB9
42,OB,77263,NaN,2.77,0.72,0.81,0.21,0.08,0.01,0.01,0.03,0.37,8.28,10.23
54,NC,63959,NaN,2.09,0.43,0.45,0.05,0.00,0.01,0.00,0.00,0.26,6.03,7.02
74,NC,61295,NaN,3.26,0.52,0.64,0.08,0.00,0.26,0.05,0.00,0.20,6.17,3.99
82,SK,60841,NaN,15.77,3.26,4.08,0.74,0.11,0.43,0.63,0.08,0.33,8.07,6.16
99,LT,64021,NaN,14.93,2.50,3.96,0.91,0.05,0.69,0.36,0.66,0.31,6.81,4.59


In [33]:
team = []
for i in batter_pred.P_ID:
    t = batter[(batter['P_ID']==i)&(batter['GDAY_DS'].dt.year==2020)]['T_ID'].unique().item()
    team.append(t)
batter_pred['T_ID'] = team

In [34]:
batter_pred.head()

,T_ID,P_ID,TB_SC,PA-AB,AB,RUN,RBI,SH+SF,KK,SB_trial,BABIP
30,NC,62907,NaN,0.37,4.01,0.73,0.45,0.08,0.38,0.07,0.35
31,NC,62934,NaN,0.14,2.08,0.31,0.33,0.03,0.37,0.26,0.15
33,HT,64646,NaN,0.21,3.74,0.61,0.28,0.03,0.46,0.14,0.31
48,NC,78122,NaN,0.14,1.58,0.19,0.26,0.05,0.23,0.00,0.24
53,KT,64006,NaN,0.20,3.07,0.24,0.34,0.07,0.82,0.20,0.17


# 4. TB_SC 명목형 변수 최빈값으로 채워주기 

- T와 B의 값이 동일한 CASE: 1/2확률로 random으로 T B 부여

In [35]:
who = []
for i in pitcher_pred.P_ID:
    if pitcher[pitcher['P_ID']==i]['TB_SC'].nunique() > 1: #T B 두 종류인 경우 
        num1 = pitcher[pitcher['P_ID']==i]['TB_SC'].value_counts()[0]
        num2 = pitcher[pitcher['P_ID']==i]['TB_SC'].value_counts()[1]
        if num1==num2 : who.append(i)
        elif num1 > num2 : pitcher_pred.loc[pitcher_pred['P_ID']==i,'TB_SC'] = pitcher[pitcher['P_ID']==i]['TB_SC'].value_counts().index[0]
        else : pitcher_pred.loc[pitcher_pred['P_ID']==i,'TB_SC'] = pitcher[pitcher['P_ID']==i]['TB_SC'].value_counts().index[1]
            
    if pitcher[pitcher['P_ID']==i]['TB_SC'].nunique() == 1: # 한 종류인 경우
        pitcher_pred.loc[pitcher_pred['P_ID']==i,'TB_SC'] = pitcher[pitcher['P_ID']==i]['TB_SC'].value_counts().index.item()

In [36]:
# T,B 값이 동일한 경우 랜덤으로 지정
for i in who:
    rand = random.randint(0,1)
    if rand == 0 : pitcher_pred.loc[pitcher_pred['P_ID']==i,'TB_SC'] = 'T'
    if rand == 1 : pitcher_pred.loc[pitcher_pred['P_ID']==i,'TB_SC'] = 'B'

In [37]:
who = []
for i in batter_pred.P_ID:
    if batter[batter['P_ID']==i]['TB_SC'].nunique() > 1: #T B 두 종류인 경우 
        num1 = batter[batter['P_ID']==i]['TB_SC'].value_counts()[0]
        num2 = batter[batter['P_ID']==i]['TB_SC'].value_counts()[1]
        if num1==num2 : who.append(i)
        elif num1 > num2 : batter_pred.loc[batter_pred['P_ID']==i,'TB_SC'] = batter[batter['P_ID']==i]['TB_SC'].value_counts().index[0]
        else : batter_pred.loc[batter_pred['P_ID']==i,'TB_SC'] = batter[batter['P_ID']==i]['TB_SC'].value_counts().index[1]
            
    if batter[batter['P_ID']==i]['TB_SC'].nunique() == 1: # 한 종류인 경우
        batter_pred.loc[batter_pred['P_ID']==i,'TB_SC'] = batter[batter['P_ID']==i]['TB_SC'].value_counts().index.item()

In [38]:
# T,B 값이 동일한 경우 랜덤으로 지정
for i in who:
    rand = random.randint(0,1)
    if rand == 0 : batter_pred.loc[batter_pred['P_ID']==i,'TB_SC'] = 'T'
    if rand == 1 : batter_pred.loc[batter_pred['P_ID']==i,'TB_SC'] = 'B'

In [39]:
batter_pred.head()

,T_ID,P_ID,TB_SC,PA-AB,AB,RUN,RBI,SH+SF,KK,SB_trial,BABIP
30,NC,62907,B,0.37,4.01,0.73,0.45,0.08,0.38,0.07,0.35
31,NC,62934,B,0.14,2.08,0.31,0.33,0.03,0.37,0.26,0.15
33,HT,64646,T,0.21,3.74,0.61,0.28,0.03,0.46,0.14,0.31
48,NC,78122,B,0.14,1.58,0.19,0.26,0.05,0.23,0.00,0.24
53,KT,64006,T,0.20,3.07,0.24,0.34,0.07,0.82,0.20,0.17


In [40]:
pitcher_pred.head()

,T_ID,P_ID,TB_SC,INN2,PA-AB,H1,H2,H3,HR,SB_SR,WP,BABIP,KK9,BB9
42,OB,77263,B,2.77,0.72,0.81,0.21,0.08,0.01,0.01,0.03,0.37,8.28,10.23
54,NC,63959,B,2.09,0.43,0.45,0.05,0.00,0.01,0.00,0.00,0.26,6.03,7.02
74,NC,61295,T,3.26,0.52,0.64,0.08,0.00,0.26,0.05,0.00,0.20,6.17,3.99
82,SK,60841,T,15.77,3.26,4.08,0.74,0.11,0.43,0.63,0.08,0.33,8.07,6.16
99,LT,64021,T,14.93,2.50,3.96,0.91,0.05,0.69,0.36,0.66,0.31,6.81,4.59


- 각 선수별 x 예측 값

In [41]:
batter_pred.reset_index(drop = True , inplace = True)

In [42]:
pitcher_pred.reset_index(drop = True, inplace  = True)

In [43]:
batter_pred.to_csv("test_data\private_batter_test.csv")
pitcher_pred.to_csv("test_data\private_pitcher_test.csv")